# Sentiment Classification with ThirdAI's Universal Deep Transformer

You can immediately run a version of this notebook in your browser on Google Colab at the following link:

https://githubtocolab.com/ThirdAILabs/Demos/blob/main/SentimentAnalysis.ipynb

This notebook uses an activation key that will only work with this demo. If you want to try us out on your own dataset, you can obtain a free trial license at the following link: https://www.thirdai.com/try-bolt/

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install datasets==2.6.1 

import thirdai
thirdai.activate("KVN9-JVRJ-FAAN-4UPE-KVXT-KV4F-LMWK-CM9E")

# Dataset Download

In the next cell, we will process an Amazon review sentiment analysis dataset from the HuggingFace datasets library. This dataset consists of product review texts with associated binary labels indicated if the review is positive or negative.

In [ ]:
import pandas as pd
from datasets import load_dataset
from thirdai.demos import to_udt_input_batch

def load_data(output_filename, split, return_inference_batch=False):
    data = load_dataset('amazon_polarity')
    
    df = pd.DataFrame(data[split])
    df = df[['title', 'label']]    
    df.to_csv(output_filename, index=False, sep='\t')
    
    if return_inference_batch:
        inference_batch = to_udt_input_batch(df[["title"]].sample(frac=1).iloc[:5])
        return inference_batch

train_filename = "amazon_polarity_train.csv"
test_filename = "amazon_polarity_test.csv"

load_data(train_filename, split='train')
inference_batch = load_data(test_filename, split='test', return_inference_batch=True)

# UDT Initialization
We can now create a UDT model by passing in the types of each column in the dataset and the target column we want to be able to predict.

In [ ]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "title": bolt.types.text(),
        "label": bolt.types.categorical()
    },
    target="label",
    n_target_classes=2,
    delimiter='\t',
)

# Training
We can now train our UDT model with just two lines! Feel free to customize the number of epochs and the learning rate; we have chosen values that give good convergence.

In [ ]:
model.train(train_filename, epochs=5, learning_rate=0.01, metrics=["categorical_accuracy"])

# Evaluation
Evaluating the performance of the UDT model is also just two lines!

In [ ]:
model.evaluate(test_filename, metrics=["categorical_accuracy"]);

# Saving and Loading
Saving and loading a trained UDT model to disk is also extremely straight forward.

In [ ]:
save_location = "sentiment_analysis.model"

# Saving
model.save(save_location)

# Loading
model = bolt.UniversalDeepTransformer.load(save_location)

# Testing Predictions
The evaluation method is great for testing, but it requires labels, which don't exist in a production setting. We also have a predict method that can take in an in-memory batch of rows or a single row (without the target column), allowing easy integration into production pipelines. Note that UDT can perform inference with **sub-millisecond** latency

In [ ]:
%%time
import numpy as np

prediction_batch = model.predict_batch(inference_batch)
class_names = ["Positive" if model.class_name(class_id) == 1 else 0 
               for class_id in np.argmax(prediction_batch, axis=1)]

print("Batch Prediction Results")
for input_sample, class_name in zip(inference_batch, class_names):
    print("Input:", input_sample, "Prediction:", class_name)